# Preparing Collected Data

In this notebook we'll go through the process of preparing the collected article data. The steps outlined here will allow for reproducing the results in production. The main interest here is to normalize the data in such a way that it can be easily utilized in exploration and modeling in production.

## Imports

These are all the modules that we'll need to run the code in this notebook.

In [1]:
import numpy as np
import pandas as pd

## Pulling the Training Data

A set of article data has been set aside for the purposes of data analysis and model training. Here I'll load that data and take a look at what features we're working with.

In [ ]:
df = pd.read_csv('articles_with_topic_label.csv')